<h1 align=center style="font-family: 'Times New Roman', sans-serif; color: darkblue">Inteligencia Artificial</h1>
<h2 align=center style="font-family: 'Times New Roman', sans-serif; color: darkblue">Practica Grupal</h2>
<h3 align=center style="font-family: 'Times New Roman', sans-serif; color: darkblue">Jhosua Callejas Ramos</h3>

<h3><p style="color: darkblue">Importacion de librerias </p></h3>

In [28]:
import pandas as pd
import requests
import glob
from io import BytesIO
import numpy as np
import os
import shutil
import pprint
import json

In [29]:
from fastai import *
from fastai.vision import *

<h3><p style="color: darkblue">Funcion para leer y procesar las imágenes de manera más eficiente y usamos tqdm para monitorear el progreso</p></h3>

<h3><p style="color: darkblue">Especificamos los directorios de los datos

In [32]:
base_path = 'food-101'
path = os.path.join(base_path, 'images')
meta_path = os.path.join(base_path, 'meta')

train_path = os.path.join(meta_path, 'train.txt')  
test_path = os.path.join(meta_path, 'test.txt')


<h3><p style="color: darkblue">Cargamos y redimensionamos los datos de train y test

In [33]:
data = ImageDataBunch.from_folder(path, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=8, bs=64).normalize(imagenet_stats)

NameError: name 'ImageDataBunch' is not defined

In [34]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

AttributeError: module 'fastai.vision.data' has no attribute 'classes'

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(40)


In [6]:
bs = 64
img_size = 224

In [18]:

from fastai.vision import *
from fastai.metrics import error_rate
# defining the data augmentation
ds_tfms = [RandTransform(tfm=TfmCrop(crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}),
           RandTransform(tfm=TfmCoord(symmetric_warp), kwargs={'magnitude': (-0.2, 0.2)}, p=0.75, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmAffine(rotate), kwargs={'degrees': (-40, 40)}, p=0.75, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmAffine(flip_affine), kwargs={}, p=0.5, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmAffine(zoom), kwargs={'scale': (1.0, 1.4), 'row_pct': (0, 1), 'col_pct': (0, 1)}, p=0.75, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmLighting(brightness), kwargs={'change': (0.35, 0.65)}, p=0.75, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmLighting(contrast), kwargs={'scale': (0.7, 1.43)}, p=0.75, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmCoord(jitter), kwargs={'magnitude': (-0.01, 0.01)}, p=0.3, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmCoord(skew), kwargs={'direction': (0, 7), 'magnitude': (0.2)}, p=0.75, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmAffine(squish), kwargs={'scale': (0.42, 2.4), 'row_pct': (0, 1), 'col_pct': (0, 1)}, p=0.75, resolved={}, do_run=True, is_random=True),
           RandTransform(tfm=TfmCrop(crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1)}, p=1.0, resolved={}, do_run=True, is_random=True)]


NameError: name 'TfmCrop' is not defined

<h3><p style="color: darkblue">Convertimos las listas en arrays de NumPy

In [22]:
from fastai.vision.all import *
data = (ImageList.from_df(df=train_df, path=path/'images', cols=1)
        .split_by_rand_pct(0.2)
        .label_from_df(cols=0)
        .transform(ds_tfm, size=img_size)
        .databuch(bs=bs)
        .normalize(imagenet_stats)
       )


NameError: name 'ImageList' is not defined

<h3><p style="color: darkblue">Verificar las dimensiones y tamaños de los datos

In [23]:
pip install --upgrade efficientnet-pytorch

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=aef4ae3aa56be8808ce2ae8ff7927b6f23acec152707ea9ee0e23b3506ec978c
  Stored in directory: c:\users\jhosu\appdata\local\pip\cache\wheels\29\16\24\752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
Successfully built efficientnet-pytorch
Note: you may need to restart the kernel to use updated packages.


In [24]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained("efficientnet-b0")


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\JHOSU/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth
100%|█████████████████████████████████████████████████████████████████████████████| 20.4M/20.4M [00:00<00:00, 40.6MB/s]

Loaded pretrained weights for efficientnet-b0


In [27]:
from fastai.vision.all import *
from efficientnet_pytorch import EfficientNet
from functools import partial

top_5 = partial(top_k_accuracy, k=5)

learn = Learner(data, model, metrics=[accuracy, top_5], callback_fns=ShowGraph).to_fp16()
learn.split(lambda m: (model._conv_head,))

learn.lr_find()
learn.recorder.plot(suggestion=True)



NameError: name 'ShowGraph' is not defined

<h3><p style="color: darkblue">Normalizamos los datos

In [38]:
# Normalizar los valores de los píxeles
train_images = train_images / 255.0
test_images = test_images / 255.0

TypeError: unsupported operand type(s) for /: 'list' and 'float'

<h3><p style="color: darkblue">Convertir las etiquetas a valores numéricos

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

train_labels_one_hot = keras.utils.to_categorical(train_labels_encoded, num_classes=len(label_encoder.classes_))
test_labels_one_hot = keras.utils.to_categorical(test_labels_encoded, num_classes=len(label_encoder.classes_))


<h3><p style="color: darkblue">Definimos y compilar el modelo</p></h3>

In [ ]:
# Cargar el modelo VGG16 preentrenado sin las capas superiores
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar todas las capas en el modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Crear el modelo completo
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

<h3><p style="color: darkblue">Entrenamos el modelo</p></h3>

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
        keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
    ]
)


<h3><p style="color: darkblue">Evaluamos el modelo</p></h3>

In [ ]:
model.evaluate(test_images, test_labels_one_hot)


<h3><p style="color: darkblue">Guardamos el modelo</p></h3>

In [ ]:
model.save('food101_model.h5')